# 0.0 Imports 

In [68]:
from prometheus_api_client import PrometheusConnect
from datetime import datetime
import pandas as pd
import os
import jornadas
import numpy as np

# 1.0 Extrator

In [69]:
def mean_requests_max_nginx(app, data_inicio, data_fim):
    
    # criando dataframe com a quantidade de reqs/s, data e posição na tabela
    request_count_nginx = pd.DataFrame()
    for posicao, numero in enumerate(list(prom.custom_query_range(query='sum(rate(nginx_ingress_controller_requests{service="' + app + '", status="200"}[1m]))', start_time=datetime.strptime(data_inicio, '%Y/%m/%d %H:%M'), end_time=datetime.strptime(data_fim, '%Y/%m/%d %H:%M'), step='1m'))[0]['values']):
        
        linha = pd.DataFrame({
            'posicao': [posicao],
            'data': numero[0],
            'valor': numero[1]
        })

        request_count_nginx = pd.concat([request_count_nginx, linha])


    # identificando pico de requests/s dentro do período de tempo passado e tirando uma média de reqs/s dos últimos 10 min a partir do pico 
    valores_pico = pd.DataFrame()
    for n in range(len(request_count_nginx)):
        if (request_count_nginx['posicao'].iloc[n] >= int(request_count_nginx[request_count_nginx['valor'] == max(request_count_nginx['valor'])]['posicao'].iloc[0]) - 10) & (request_count_nginx['posicao'].iloc[n] <= int(request_count_nginx[request_count_nginx['valor'] == max(request_count_nginx['valor'])]['posicao'].iloc[0])):
            
            linha1 = pd.DataFrame({
                'valor': [float(request_count_nginx['valor'].iloc[n])]
            }) 

            valores_pico = pd.concat([valores_pico, linha1])

    return round(valores_pico['valor'].mean() * 60) 

In [70]:
# transformando listas de dicts em dataframes
df = pd.concat([pd.DataFrame(data=jornadas.compra), pd.DataFrame(data=jornadas.b2b), pd.DataFrame(data=jornadas.figital)]).reset_index()

# criando uma lista com os nomes dos servidores 
server = list(df['server'].copy())

# limpando a coluna "index" e a coluna "server"
df = df.drop(['index', 'server'], axis=1)

df['bateria'] = np.nan

## 1.1 Extração

In [71]:
qtd_baterias = input('Quantidade de bateria no teste')


if int(qtd_baterias) >= int(1):
    data_inicio = input('Data Hora de Início da Bateria 1')
    data_fim = input('Data Hora de Término da Bateria 1')

if int(qtd_baterias) >= int(2):
    data_inicio2 = input('Data Hora de Início da Bateria 2')
    data_fim2 = input('Data Hora de Término da Bateria 2') 

if int(qtd_baterias) >= int(3):
    data_inicio3 = input('Data Hora de Início da Bateria 3')
    data_fim3 = input('Data Hora de Término da Bateria 3')



for i in range(int(qtd_baterias)):

    if int(i+1) == 2:
        data_inicio = data_inicio2
        data_fim = data_fim2

    if int(i+1) == 3:
        data_inicio = data_inicio3
        data_fim = data_fim3
    
    for numero, coluna in enumerate(df):
        if numero > 2:
            for num, app in enumerate(df[coluna]):
                prom = PrometheusConnect(url =f"{server[num]}", disable_ssl=True)
                try:
                    df[coluna].iloc[num] = mean_requests_max_nginx(str(app), data_inicio, data_fim)

                except:
                    df[coluna].iloc[num] = ''

    if int(i+1) == 1:
        df['bateria'] = 1
        bateria1 = df.copy()
    
    if int(i+1) == 2:
        df['bateria'] = 2
        bateria2 = df.copy()
    
    if int(i+1) == 3:
        df['bateria'] = 3
        bateria3 = df.copy()
    
    # transformando listas de dicts em dataframes
    df = pd.concat([pd.DataFrame(data=jornadas.compra), pd.DataFrame(data=jornadas.b2b), pd.DataFrame(data=jornadas.figital)]).reset_index()

    # criando uma lista com os nomes dos servidores 
    server = list(df['server'].copy())

    # limpando a coluna "index" e a coluna "server"
    df = df.drop(['index', 'server'], axis=1)

    df['bateria'] = np.nan

if int(qtd_baterias) == 1:
    df = bateria1.copy()

if int(qtd_baterias) == 2:
    df = pd.concat([bateria1, bateria2])

if int(qtd_baterias) == 3:
    df = pd.concat([bateria1, bateria2, bateria3])

C:\Users\4960021521\AppData\Local\Temp\ipykernel_7248\3501622777.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[coluna].iloc[num] = mean_requests_max_nginx(str(app), data_inicio, data_fim)
C:\Users\4960021521\AppData\Local\Temp\ipykernel_7248\3501622777.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[coluna].iloc[num] = mean_requests_max_nginx(str(app), data_inicio, data_fim)
C:\Users\4960021521\AppData\Local\Temp\ipykernel_7248\3501622777.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pa

## 1.2 Resultado

In [72]:
df

,jornada,tribo,produto,reqs_cb,reqs_ex,reqs_pf,reqs_corp,bateria
0,Compra,vitrine,carrinho,767,159,204,,1
1,Compra,carrinho-front,apollo,294,35,60,,1
2,Compra,check,checkout,591,90,132,,1
3,b2b,parcerias,checkout-front,341,24,33,,1
4,b2b,parcerias,kong_proxy,4427,1345,849,,1
5,b2b,parcerias,kong,,,,109,1
6,teste,teste,teste,341,24,33,,1
7,teste,teste,teste,,,,,1
8,teste,teste,teste,,,,109,1
0,Compra,vitrine,carrinho,731,295,332,,2
